In [2]:
from keras_tuner import HyperModel
import keras_tuner as kt

import numpy as np
import pandas as pd
#!pip install keras_tuner
#import keras_tuner

import keras
from keras import models, layers, metrics, Sequential
from keras.layers import Dense, BatchNormalization, Dropout
import tensorflow as tf
from keras.regularizers import l1,l2,l1_l2
from sklearn.metrics import mean_squared_error
from tensorflow.keras.layers import Input, Dense, Multiply, Concatenate
from tensorflow.keras.models import Model

import tensorflow as tf
import random

import keras_tuner
from keras_tuner import HyperModel
import keras_tuner as kt

In [1]:
# Import Dataset
import pandas as pd
path = 'dataset/'
firm_df = pd.read_pickle(path+'firm_df49.pkl')
firm_df.index = pd.to_datetime(firm_df.index).to_period('M')

# macro-economic variables
# There is one features which has 37 missing values but before year 1980
macro_df = pd.read_pickle(path+'macro_df.pkl')
macro_df.index = pd.to_datetime(macro_df.index, format='%m/%d/%Y')
macro_df.rename(columns={"ep": "ep_macro"}, inplace=True)
macro_index = macro_df.index[1:]
macro_new = macro_df.iloc[:-1, :]
macro_new.index = macro_index
macro_new.index = macro_new.index.to_period('M')

# Scale macro-features
from sklearn.preprocessing import MinMaxScaler
#scaler = MinMaxScaler(feature_range=(-0.5, 0.5))
#macro_scaled = scaler.fit_transform(macro_new)
#macro_scaled_df = pd.DataFrame(macro_scaled, columns=macro_new.columns)
#macro_scaled_df.index = macro_new.index

# merged data 1
merged_df = pd.merge(firm_df, macro_new, left_on='jdate', right_on='sasdate', how='inner')
merged_df.index = firm_df.index

In [3]:
char_core = ['acc','agr','beta','bm','cash','cashpr','cfp','chatoia','chcsho','chfeps','chinv',
       'chmom','chpmia','chtx','currat','depr','dy','ear','ep','gma','grcapx','grltnoa',
       'ill','indmom','invest','lev','lgr','maxret','mom12m','mom1m','mom36m','mve','nincr',
       'orgcap','pchgm_pchsale','pchsale_pchinvt','pchsale_pchrect','pchsale_pchxsga',
       'retvol','roaq','roavol','roeq','salecash','saleinv','sgr','sp','std_dolvol','std_turn','turn']

info_list = ['fyear','year','jyear','permno','ticker','comnam','exchcd','exchname','siccd',
       'indname','size_class','mve_m','rf','ret','ret_adj','ret_ex','ret_adj_ex',]

macro_col = ['RPI', 'W875RX1', 'DPCERA3M086SBEA', 'CMRMTSPLx', 'RETAILx','INDPRO', 'IPFPNSS', 'IPFINAL', 'IPCONGD', 'IPDCONGD', 'IPNCONGD',
       'IPBUSEQ', 'IPMAT', 'IPDMAT', 'IPNMAT', 'IPMANSICS', 'IPB51222S','IPFUELS', 'CUMFNS', 'HWI', 'HWIURATIO', 'CLF16OV', 'CE16OV',
       'UNRATE', 'UEMPMEAN', 'UEMPLT5', 'UEMP5TO14', 'UEMP15OV','UEMP15T26', 'UEMP27OV', 'CLAIMSx', 'PAYEMS', 'USGOOD',
       'CES1021000001', 'USCONS', 'MANEMP', 'DMANEMP', 'NDMANEMP','SRVPRD', 'USTPU', 'USWTRADE', 'USTRADE', 'USFIRE', 'USGOVT',
       'CES0600000007', 'AWOTMAN', 'AWHMAN', 'HOUST', 'HOUSTNE','HOUSTMW', 'HOUSTS', 'HOUSTW', 'PERMIT', 'PERMITNE', 'PERMITMW',
       'PERMITS', 'PERMITW', 'AMDMNOx', 'ANDENOx', 'AMDMUOx', 'BUSINVx','ISRATIOx', 'M1SL', 'M2SL', 'M2REAL', 'BOGMBASE', 'TOTRESNS',
       'NONBORRES', 'BUSLOANS', 'REALLN', 'NONREVSL', 'CONSPI', 'S&P 500','S&P: indust', 'S&P div yield', 'S&P PE ratio', 'FEDFUNDS',
       'CP3Mx', 'TB3MS', 'TB6MS', 'GS1', 'GS5', 'GS10', 'AAA', 'BAA','COMPAPFFx', 'TB3SMFFM', 'TB6SMFFM', 'T1YFFM', 'T5YFFM', 'T10YFFM',
       'AAAFFM', 'BAAFFM', 'TWEXAFEGSMTHx', 'EXSZUSx', 'EXJPUSx','EXUSUKx', 'EXCAUSx', 'WPSFD49207', 'WPSFD49502', 'WPSID61',
       'WPSID62', 'OILPRICEx', 'PPICMM', 'CPIAUCSL', 'CPIAPPSL','CPITRNSL', 'CPIMEDSL', 'CUSR0000SAC', 'CUSR0000SAD',
       'CUSR0000SAS', 'CPIULFSL', 'CUSR0000SA0L2', 'CUSR0000SA0L5','PCEPI', 'DDURRG3M086SBEA', 'DNDGRG3M086SBEA', 'DSERRG3M086SBEA',
       'CES0600000008', 'CES2000000008', 'CES3000000008', 'UMCSENTx','DTCOLNVHFNM', 'DTCTHFNM', 'INVEST', 'VIXCLSx', 'dp', 'ep_macro', 'b/m',
       'ntis', 'tbl', 'tms', 'dfy', 'svar']

macro_core =  ['dp', 'ep_macro', 'b/m','ntis', 'tbl', 'tms', 'dfy', 'svar']

def datasplit(df, dataset):
    df_train = df[df.index < pd.Period((str(1995)+"-1"),freq='M')]
    df_val = df[(df.index >= pd.Period((str(1995)+"-1"),freq='M')) & (df.index < pd.Period((str(2000)+"-1"),freq='M'))]
    df_test = df[df.index >= pd.Period((str(2000)+"-1"),freq='M')]
    if dataset == "firm":
        X_train = df_train[char_core]
        X_val = df_val[char_core]
        X_test = df_test[char_core]
    elif dataset == 'macro':
        X_train = df_train[macro_col]
        X_val = df_val[macro_col]
        X_test = df_test[macro_col]
    elif dataset == "firm_macro":
        X_train = df_train[char_core+macro_col]
        X_val = df_val[char_core+macro_col]
        X_test = df_test[char_core+macro_col]
    elif dataset == "firm_macro8":
        X_train = df_train[char_core+macro_core]
        X_val = df_val[char_core+macro_core]
        X_test = df_test[char_core+macro_core] 
    y_train = df_train['predicted_return']
    y_val = df_val['predicted_return']
    y_test = df_test['predicted_return']
    return (X_train, y_train, X_val, y_val, X_test, y_test, )

In [4]:
def my_metric_fn(y_true, y_pred):
    num = tf.reduce_mean(tf.square(y_true - y_pred))
    den = tf.reduce_mean(tf.square(y_true - tf.zeros_like(y_true)))
    return 1 - num / den

def call_existing_code(input_value):
    L = 4
    l1_reg= 0.
    l2_reg= 0.01
    dropout =0.1
    lr =0.001
    acti='relu'
    model = Sequential()
    model.add(Dense(units=64, input_dim=input_value, kernel_regularizer=tf.keras.regularizers.l1_l2(l1=l1_reg, l2=l2_reg), activation=acti))
    model.add(Dropout(dropout))
    model.add(BatchNormalization())
    for i in range(L - 1):
        model.add(Dense(int(32/2**i), kernel_regularizer=tf.keras.regularizers.l1_l2(l1=l1_reg, l2=l2_reg), activation=acti))
        model.add(Dropout(dropout))
        model.add(BatchNormalization())
    model.add(Dense(1))
    optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=lr)
    model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=[my_metric_fn])
    return model

In [ ]:
def build4_NNc(n_inputs,):
    l1_reg = 0.
    l2_reg = 0.01
    dropout = 0.1
    lr = 0.01

    # Define the hidden layers
    input_layer = Input(shape=(n_inputs,))
    # Define hidden layers
    hidden_layer1 = Dense(64, activation ='relu', kernel_regularizer=l1_l2(l1=l1_reg,l2=l2_reg))(input_layer)
    dropout1 = Dropout(dropout)(hidden_layer1)
    hidden_layer2 = Dense(32, activation ='relu', kernel_regularizer=l1_l2(l1=l1_reg,l2=l2_reg))(hidden_layer1)
    dropout2 = Dropout(dropout)(hidden_layer2)
    hidden_layer3 = Dense(16, activation ='relu', kernel_regularizer=l1_l2(l1=l1_reg,l2=l2_reg))(hidden_layer2)
    dropout3 = Dropout(dropout)(hidden_layer3)
    hidden_layer4 = Dense(8, activation ='relu', kernel_regularizer=l1_l2(l1=l1_reg,l2=l2_reg))(hidden_layer2)
    dropout4 = Dropout(dropout)(hidden_layer4)

    # Concatenate the last hidden layer with the input layer
    concatenated_layer = Concatenate()([hidden_layer4, input_layer])
    # Define output layer
    output_layer = Dense(1,)(concatenated_layer)
    # Create the model
    model = Model(input_layer, output_layer)
    opt = tf.keras.optimizers.Adam(learning_rate=lr)
    model.compile(loss='mean_squared_error', optimizer=opt,metrics=[my_metric_fn])
    return model

## Model 4a: 49 +8

In [51]:
df_test = merged_df[firm_df.index >= pd.Period((str(1999)+"-12"),freq='M')]
y_test = df_test['predicted_return']
x_test = df_test[char_core+macro_core]
print(x_test.shape, y_test.shape)

(90479, 57) (90479,)


In [ ]:
seed_list = [458, 165, 530, 564, 590, 560, 829, 170, 376, 176]
yhat_df = pd.DataFrame()

for random_seed in seed_list:
    random.seed(random_seed)
    np.random.seed(random_seed)
    tf.random.set_seed(random_seed)
    
    y_hat = []
    for i in range(len(y_test.index.unique())):
        train_date = y_test.index.unique()[i]
        if train_date == pd.Period((str(2022)+"-"+str(5)),freq='M'):
            break
        test_date = y_test.index.unique()[i+1]
        X_train = x_test[x_test.index == train_date]
        Y_train = y_test[y_test.index == train_date]
        X_test = x_test[x_test.index == test_date]
        Y_test = y_test[y_test.index == test_date]

        model = build4_NNc(n_inputs= X_train.shape[1])
        model.fit(X_train.values, Y_train.values,
              epochs=200, batch_size=32, verbose=0,
              callbacks=[keras.callbacks.EarlyStopping(monitor='loss', mode='min', verbose=1, patience=2)])
        yhat_NN = model.predict(X_test.values).reshape(-1)
        for t in yhat_NN:
            y_hat.append(t)
    yhat_df[random_seed] = y_hat
    print(yhat_df.shape)

In [54]:
Y_2 = df_test[df_test.index > pd.Period((str(1999)+"-"+str(12)),freq='M')]
y_predict = yhat_df.mean(axis=1).values
y_real = Y_2['predicted_return']
a = np.mean(np.square(y_predict -  y_real))
b = np.mean(np.square(y_real))
R2 = 1-a/b
print(f"R^2 is {R2}")

path_y = 'predict/'
yhat_df.to_pickle(path_y+"model4a49+8.pkl")

R^2 is -0.38899586176322254


## Model 4b: 49 +134

In [55]:
df_test = merged_df[firm_df.index >= pd.Period((str(1999)+"-12"),freq='M')]
y_test = df_test['predicted_return']
x_test = df_test[char_core+macro_col]
print(x_test.shape, y_test.shape)

(90479, 183) (90479,)


In [ ]:
seed_list = [1120]
yhat_df = pd.DataFrame()

for random_seed in seed_list:
    random.seed(random_seed)
    np.random.seed(random_seed)
    tf.random.set_seed(random_seed)
    
    y_hat = []
    for i in range(len(y_test.index.unique())):
        train_date = y_test.index.unique()[i]
        if train_date == pd.Period((str(2022)+"-"+str(5)),freq='M'):
            break
        test_date = y_test.index.unique()[i+1]
        X_train = x_test[x_test.index == train_date]
        Y_train = y_test[y_test.index == train_date]
        X_test = x_test[x_test.index == test_date]
        Y_test = y_test[y_test.index == test_date]

        model = build4_NNc(n_inputs= X_train.shape[1])
        model.fit(X_train.values, Y_train.values,
              epochs=200, batch_size=32, verbose=0,
              callbacks=[keras.callbacks.EarlyStopping(monitor='loss', mode='min', verbose=1, patience=2)])
        yhat_NN = model.predict(X_test.values).reshape(-1)
        for t in yhat_NN:
            y_hat.append(t)
    yhat_df[random_seed] = y_hat
    print(yhat_df.shape)

In [70]:
Y_2 = df_test[df_test.index > pd.Period((str(1999)+"-"+str(12)),freq='M')]
y_predict = yhat_df.mean(axis=1).values
y_real = Y_2['predicted_return']
a = np.mean(np.square(y_predict -  y_real))
b = np.mean(np.square(y_real))
R2 = 1-a/b
print(f"R^2 is {R2}")


R^2 is -0.41035475951794886


In [72]:
Y_2 = df_test[df_test.index > pd.Period((str(1999)+"-"+str(12)),freq='M')]
y_predict = yhat_df.mean(axis=1).values
y_real = Y_2['predicted_return']
a = np.mean(np.square(y_predict -  y_real))
b = np.mean(np.square(y_real))
R2 = 1-a/b
print(f"R^2 is {R2}")

path_y = 'predict/'
yhat_df.to_pickle(path_y+"model4b49+134.pkl")

R^2 is -0.41035475951794886


## Model 4c: 49 Firm-levl Characteristics 

In [60]:
def build4_NNc(n_inputs,):
    l1_reg = 0.
    l2_reg = 0.01
    dropout = 0.1
    lr = 0.01

    # Define the hidden layers
    input_layer = Input(shape=(n_inputs,))
    # Define hidden layers
    hidden_layer1 = Dense(64, activation ='relu', kernel_regularizer=l1_l2(l1=l1_reg,l2=l2_reg))(input_layer)
    dropout1 = Dropout(dropout)(hidden_layer1)
    hidden_layer2 = Dense(32, activation ='relu', kernel_regularizer=l1_l2(l1=l1_reg,l2=l2_reg))(hidden_layer1)
    dropout2 = Dropout(dropout)(hidden_layer2)
    hidden_layer3 = Dense(16, activation ='relu', kernel_regularizer=l1_l2(l1=l1_reg,l2=l2_reg))(hidden_layer2)
    dropout3 = Dropout(dropout)(hidden_layer3)
    hidden_layer4 = Dense(8, activation ='relu', kernel_regularizer=l1_l2(l1=l1_reg,l2=l2_reg))(hidden_layer2)
    dropout4 = Dropout(dropout)(hidden_layer4)

    # Concatenate the last hidden layer with the input layer
    concatenated_layer = Concatenate()([hidden_layer4, input_layer])
    # Define output layer
    output_layer = Dense(1,)(concatenated_layer)
    # Create the model
    model = Model(input_layer, output_layer)
    opt = tf.keras.optimizers.Adam(learning_rate=lr)
    model.compile(loss='mean_squared_error', optimizer=opt,metrics=[my_metric_fn])
    return model

In [61]:
df_test = firm_df[firm_df.index >= pd.Period((str(1999)+"-12"),freq='M')]
y_test = df_test['predicted_return']
x_test = df_test[char_core]
print(x_test.shape, y_test.shape)

(90479, 49) (90479,)


In [ ]:
seed_list = [458, 165, 530, 564, 590, 560, 829, 170, 376, 176]
yhat_df = pd.DataFrame()

for random_seed in seed_list:
    random.seed(random_seed)
    np.random.seed(random_seed)
    tf.random.set_seed(random_seed)
    
    y_hat = []
    for i in range(len(y_test.index.unique())):
        train_date = y_test.index.unique()[i]
        if train_date == pd.Period((str(2022)+"-"+str(5)),freq='M'):
            break
        test_date = y_test.index.unique()[i+1]
        X_train = x_test[x_test.index == train_date]
        Y_train = y_test[y_test.index == train_date]
        X_test = x_test[x_test.index == test_date]
        Y_test = y_test[y_test.index == test_date]

        model = build4_NNc(n_inputs= X_train.shape[1])
        model.fit(X_train.values, Y_train.values,
              epochs=200, batch_size=32, verbose=0,
              callbacks=[keras.callbacks.EarlyStopping(monitor='loss', mode='min', verbose=1, patience=2)])
        yhat_NN = model.predict(X_test.values).reshape(-1)
        for t in yhat_NN:
            y_hat.append(t)
    yhat_df[random_seed] = y_hat
    print(yhat_df.shape)

In [65]:
Y_2 = df_test[df_test.index > pd.Period((str(1999)+"-"+str(12)),freq='M')]
y_predict = yhat_df.mean(axis=1).values
y_real = Y_2['predicted_return']
a = np.mean(np.square(y_predict -  y_real))
b = np.mean(np.square(y_real))
R2 = 1-a/b
print(f"R^2 is {R2}")

path_y = 'predict/'
yhat_df.to_pickle(path_y+"model4c49.pkl")

R^2 is -0.35747186794062147


X_train, y_train, X_val, y_val, X_test, y_test = datasplit(firm_df, dataset = "firm")
print(X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape, y_test.shape)